In [1]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./trajopt/constraint.jl")
include("./trajopt/costfunction.jl")
include("./trajopt/scaling.jl")
include("./trajopt/scp.jl")
include("./trajopt/discretize.jl")

LoadError: LoadError: UndefVarError: `Rocket` not defined
in expression starting at /Users/taewankim/Documents/Code/Personal/JointSynthesisFreeFinal/trajopt/constraint.jl:126

In [2]:
ix = 12
iu = 4
N = 15
dynamics = QuadrotorDynamics()

QuadrotorDynamics(12, 4, 9, 7, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1.325, 9.81, 0.03843, 0.02719, 0.060528)

In [ ]:
v_max = 5
att_max = pi/3
att_vel_max = pi
F_max = 20
M_max = 3
rotor_const = QuadrotorConstraint(v_max,att_max,att_vel_max,F_max,M_max)
r = 5.0
h = 5.0
phase_const = QuadrotorMultiphaseConstraint(N,r,h)
# list_const = [rotor_const];
list_const = [rotor_const,phase_const];

In [ ]:
xi = [phase_const.P1[1],phase_const.P1[2],h, 0,0,0, deg2rad(0),deg2rad(0),deg2rad(0), 0,0,0]
# xf = [phase_const.P2[1],phase_const.P2[2],h, 0,0,0, deg2rad(0),deg2rad(0),deg2rad(0), 0,0,0]
xf = copy(xi)
@assert length(xi) == ix
@assert length(xf) == ix

In [ ]:
x0 = zeros(ix,N+1);
u0 = zeros(iu,N+1);
x0[3,:] .= h
u0[1,:] .= dynamics.m * dynamics.g
tf0 = 30.0;
dt0 = tf0/N*ones(N);

for i = 1:N+1
    x0[:,i] = (N-i+1)/N*xi+(i-1)/N*xf;
end

In [ ]:
function get_x0(x0,N1,N2,P1,P2)
    N_ = N2 - N1
    for i = N1:N2
        x0[1:2,i] = (N2-i)/(N_)*P1+(i-N1)/(N_)*P2;
        # println(i)
    end
    return
end
get_x0(x0,phase_const.N1,phase_const.N2,phase_const.P1,phase_const.P2)
get_x0(x0,phase_const.N2,phase_const.N3,phase_const.P2,phase_const.P3)
get_x0(x0,phase_const.N3,phase_const.N4,phase_const.P3,phase_const.P4)
get_x0(x0,phase_const.N4,phase_const.N5,phase_const.P4,phase_const.P5)
get_x0(x0,phase_const.N5,phase_const.N6,phase_const.P5,phase_const.P1)
xdes = copy(x0[1:2,:])

In [ ]:
plot(x0[1,:],x0[2,:],aspect_ratio=:equal)
plot!(x0[1,:],x0[2,:],seriestype=:scatter)

In [ ]:
xmin = zeros(ix);
xmax = [5,5,5, v_max,v_max,v_max, att_max,att_max,att_max, att_vel_max,att_vel_max,att_vel_max];
umin = zeros(iu);
umax = [F_max,M_max,M_max,M_max];
min_dt = 0.1;
max_dt = 10.0;
scaler = Scaling(xmin, xmax, umin, umax, tf0, min_dt,max_dt)
@assert max_dt * N >= tf0 

In [ ]:
max_iter = 50;
w_tf = 2*1e0;
w_c = 0.0;
w_rate = 1e1;
w_param = 0.0;
w_vc = 1e2;
w_tr::Float64 = 5*1e1;
tol_vc = 1e-6;
tol_tr = 1e-3;
tol_dyn = 1e-1;
tr_norm = "quad";
verbosity = true;
# type_tf_free = "tf_free";
type_tf_free = "five_phase";

In [ ]:
ptr = PTR(N,tf0,max_iter,dynamics,list_const,scaler,
    w_tf,w_c,w_rate,w_param,w_vc,w_tr,
    tol_vc,tol_tr,tol_dyn,
    tr_norm,verbosity,type_tf_free=type_tf_free)

In [ ]:
run(ptr,x0,u0,dt0,xi,xf,"Gurobi")

In [ ]:
# plot(x0[1,:],x0[2,:],linestyle=:dash,aspect_ratio=:equal)
plot(ptr.solution.xprop[1,:],ptr.solution.xprop[2,:],aspect_ratio=:equal,label="nominal trajectory")
plot!(ptr.solution.x[1,:],ptr.solution.x[2,:],seriestype=:scatter)
# plot!(ptr.solution.x[1,:],ptr.solution.x[2,:])

In [ ]:
plot(ptr.solution.t,ptr.solution.x[3,:],seriestype=:scatter)
plot!(ptr.solution.tprop,ptr.solution.xprop[3,:])

In [ ]:
p1 = plot(ptr.solution.t,ptr.solution.x[4,:],seriestype = :scatter,label="")
plot!(ptr.solution.tprop,ptr.solution.xprop[4,:],label="vx")
p2 = plot(ptr.solution.t,ptr.solution.x[5,:],seriestype = :scatter,label="")
plot!(ptr.solution.tprop,ptr.solution.xprop[5,:],label="vy")
p3 = plot(ptr.solution.t,ptr.solution.x[6,:],seriestype = :scatter,label="")
plot!(ptr.solution.tprop,ptr.solution.xprop[6,:],label="vz")

plot(p1,p2,p3,layout=(3,1))

In [ ]:
p1 = plot(ptr.solution.t,ptr.solution.x[7,:],seriestype = :scatter,label="")
plot!(ptr.solution.tprop,ptr.solution.xprop[7,:],label="roll")
p2 = plot(ptr.solution.t,ptr.solution.x[8,:],seriestype = :scatter,label="")
plot!(ptr.solution.tprop,ptr.solution.xprop[8,:],label="pitch")
p3 = plot(ptr.solution.t,ptr.solution.x[9,:],seriestype = :scatter,label="")
plot!(ptr.solution.tprop,ptr.solution.xprop[9,:],label="yaw")

plot(p1,p2,p3,layout=(3,1))

In [ ]:
p1 = plot(ptr.solution.t,ptr.solution.x[10,:],seriestype = :scatter,label="")
plot!(ptr.solution.tprop,ptr.solution.xprop[10,:],label="p")
p2 = plot(ptr.solution.t,ptr.solution.x[11,:],seriestype = :scatter,label="")
plot!(ptr.solution.tprop,ptr.solution.xprop[11,:],label="q")
p3 = plot(ptr.solution.t,ptr.solution.x[12,:],seriestype = :scatter,label="")
plot!(ptr.solution.tprop,ptr.solution.xprop[12,:],label="r")

plot(p1,p2,p3,layout=(3,1))

In [ ]:
p1 = plot(ptr.solution.t,ptr.solution.u[1,:],seriestype = :scatter,label="F")
plot!(ptr.solution.t,ptr.solution.u[1,:],label="")
p2 = plot(ptr.solution.t,ptr.solution.u[2,:],seriestype = :scatter,label="Mx")
plot!(ptr.solution.t,ptr.solution.u[2,:],label="")
p3 = plot(ptr.solution.t,ptr.solution.u[3,:],seriestype = :scatter,label="My")
plot!(ptr.solution.t,ptr.solution.u[3,:],label="")
p4 = plot(ptr.solution.t,ptr.solution.u[4,:],seriestype = :scatter,label="Mz")
plot!(ptr.solution.t,ptr.solution.u[4,:],label="")

plot(p1,p2,p3,p4,layout=(4,1))

In [ ]:
using JLD2, FileIO
my_dict = Dict("x" => ptr.solution.x, "u" => ptr.solution.u, "t" => ptr.solution.t)
@save "./data/quadstar_N15_traj" my_dict